# XGBoost Demand Prediction
This notebook use XGBoost ensemble to predict the inventory demand.

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_log_error as rmsle
from sklearn.metrics import root_mean_squared_error
import xgboost as xgb

## Import Datasets

**Training Dataset**
The training dataset contains the following variables:
* Client ID
* Product ID
* Week
* Sales Depot
* Sales Channel
* Route ID
* Sales Quantity and Value
* Return Quantity and Value
* Adjusted Demand (target variable)

**Test Dataset**
The test dataset containes the following variables:
* Client ID
* Product ID
* Week
* Sales Channel
* Route ID

First, we combine the training and test datasets to define a new ID variables based on unique client ID, product ID, sales Channel, route ID, and sales depot. Then, we split the training and test dataset.

In [2]:
#data set from kaggle: https://www.kaggle.com/competitions/grupo-bimbo-inventory-demand/data

#office
train = pd.read_csv("train.csv", usecols=['Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Producto_ID', 'Cliente_ID', 'Demanda_uni_equil'])
test = pd.read_csv("test.csv", usecols=['Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Producto_ID', 'Cliente_ID', 'id'])

train = train.rename(columns={'Semana': 'Week_num',
                              'Agencia_ID': 'Sales_Depot_ID',
                              'Canal_ID': 'Sales_Channel_ID',
                              'Ruta_SAK': 'Route_ID',
                              'Cliente_ID': 'Client_ID',
                              'Venta_uni_hoy': 'Sales_unit_this_week',
                              'Venta_hoy': 'Sales_this_week',
                              'Dev_uni_proxima': 'Returns_unit_next_week',
                              'Dev_proxima': 'Returns_next_week',
                              'Demanda_uni_equil': 'adjusted_demand',
                              'Producto_ID': 'Product_ID'})

test = test.rename(columns={'Semana': 'Week_num',
                            'Agencia_ID': 'Sales_Depot_ID',
                            'Canal_ID': 'Sales_Channel_ID',
                            'Ruta_SAK': 'Route_ID',
                            'Cliente_ID': 'Client_ID',
                            'Venta_uni_hoy': 'Sales_unit_this_week',
                            'Venta_hoy': 'Sales_this_week',
                            'Dev_uni_proxima': 'Returns_unit_next_week',
                            'Dev_proxima': 'Returns_next_week',
                            'Demanda_uni_equil': 'adjusted_demand',
                            'Producto_ID': 'Product_ID'})



#set a unique id for each sales depot id, sales channel id, route id, client, product combination (thanks Gemini)
combined_df = pd.concat([train,test])
combined_df['ID'] = combined_df.groupby(['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Client_ID', 'Product_ID']).ngroup()

#set a combined client ID, consisting of a unique sales depot ID, sales channel ID, route ID, and client ID
combined_df['ccid'] = combined_df.groupby(['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Client_ID']).ngroup()

#set a combined product ID, consisting of a unique sales depot ID, sales channel ID, route ID, and product ID
combined_df['cpid'] = combined_df.groupby(['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Product_ID']).ngroup()

train = combined_df.iloc[:len(train)].copy()
test = combined_df.iloc[len(train):].copy()

del combined_df


train = train.drop(columns='id')
train['adjusted_demand'] = train['adjusted_demand'].astype(int)
train = train.sort_values(by=['ID', 'Week_num']).reset_index(drop=True)

test = test.drop(columns='adjusted_demand')
test['id'] = test['id'].astype(int)
test = test.sort_values(by=['ID', 'Week_num']).reset_index(drop=True)

## Aggregate Data at Client Level and Product Level
In this step, we create various aggregate measures of demand at client level and the product level. We create these measures using data from week 3-8 to avoid data leakage.

In [ ]:
#create a dataframe of aggregate statistics for each client
testagg = train[train['Week_num'] <= 8].groupby(['ccid'], as_index=False).agg({'Product_ID':'nunique', 'adjusted_demand':['mean', 'median', 'min', 'max']})

client_stats = pd.DataFrame()

client_stats['ccid'] = testagg['ccid']
client_stats['Products'] = testagg['Product_ID']['nunique']
client_stats['adj_dem_mean'] = testagg['adjusted_demand']['mean'].round(2)
client_stats['adj_dem_median'] = testagg['adjusted_demand']['median'].astype(int)
client_stats['adj_dem_min'] = testagg['adjusted_demand']['min']
client_stats['adj_dem_max'] = testagg['adjusted_demand']['max']
client_stats['mean_pct'] = client_stats['adj_dem_mean'].rank(pct=True, method='average')

del testagg

#create a dataframe of aggregate statistics for each product
testagg = train[train['Week_num'] <= 8].groupby(['cpid'], as_index=False).agg({'Client_ID':'nunique', 'adjusted_demand':['mean', 'median', 'min', 'max']})

product_stats =  pd.DataFrame()

product_stats['cpid'] = testagg['cpid']
product_stats['Clients'] = testagg['Client_ID']['nunique']
product_stats['adj_dem_mean'] = testagg['adjusted_demand']['mean'].round(2)
product_stats['adj_dem_median'] = testagg['adjusted_demand']['median'].astype(int)
product_stats['adj_dem_min'] = testagg['adjusted_demand']['min']
product_stats['adj_dem_max'] = testagg['adjusted_demand']['max']
product_stats['median_pct'] = product_stats['adj_dem_median'].rank(pct=True, method='average')

del testagg

We merge the `client_stats` and `product_stats` dataframes with the `train` dataset.

In [4]:
# get ccid, cpid means, medians and cpid median percentage into training data
cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())].set_index('ccid')['adj_dem_mean'], index=client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())]['ccid']).to_dict()
train['ccid_mean'] = train['ccid'].map(cidmapping)

cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())].set_index('ccid')['adj_dem_median'], index=client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())]['ccid']).to_dict()
train['ccid_median'] = train['ccid'].map(cidmapping)

cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())].set_index('ccid')['adj_dem_min'], index=client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())]['ccid']).to_dict()
train['ccid_min'] = train['ccid'].map(cidmapping)

cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())].set_index('ccid')['adj_dem_max'], index=client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())]['ccid']).to_dict()
train['ccid_max'] = train['ccid'].map(cidmapping)

cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())].set_index('ccid')['mean_pct'], index=client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())]['ccid']).to_dict()
train['ccid_mean_pct'] = train['ccid'].map(cidmapping).round(3)

pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())].set_index('cpid')['adj_dem_mean'], index=product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())]['cpid']).to_dict()
train['cpid_mean'] = train['cpid'].map(pidmapping)

pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())].set_index('cpid')['adj_dem_median'], index=product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())]['cpid']).to_dict()
train['cpid_median'] = train['cpid'].map(pidmapping)

pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())].set_index('cpid')['median_pct'], index=product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())]['cpid']).to_dict()
train['cpid_median_pct'] = train['cpid'].map(pidmapping).round(3)

del cidmapping, pidmapping

train.head()

,Week_num,Sales_Depot_ID,Sales_Channel_ID,Route_ID,Client_ID,Product_ID,adjusted_demand,ID,ccid,cpid,ccid_mean,ccid_median,ccid_min,ccid_max,ccid_mean_pct,cpid_mean,cpid_median,cpid_median_pct
0,4,1110,7,3301,15766,325,1,0,0,4,5.51,4.0,1.0,23.0,0.693,1.83,1.0,0.082
1,4,1110,7,3301,15766,328,1,1,0,5,5.51,4.0,1.0,23.0,0.693,1.88,2.0,0.230
2,3,1110,7,3301,15766,1212,3,2,0,15,5.51,4.0,1.0,23.0,0.693,2.51,2.0,0.230
3,4,1110,7,3301,15766,1212,4,2,0,15,5.51,4.0,1.0,23.0,0.693,2.51,2.0,0.230
4,5,1110,7,3301,15766,1212,5,2,0,15,5.51,4.0,1.0,23.0,0.693,2.51,2.0,0.230


## Lag Variables
We use adjusted demand in week 9 as our target variables. To enrich our model, we define lag variables i.e. adjusted demand in week 6, 7, and 8.

In [5]:
# create training data based on ID in order to use lagged adjusted demand
trainIDdf = pd.DataFrame()

trainIDdf = train[['ID', 'Client_ID', 'Product_ID', 'ccid_mean', 'ccid_median', 'cpid_mean', 'cpid_median']].drop_duplicates(subset='ID', keep='first').reset_index(drop=True)

# get adjusted demand for the week for each ID 
for j in [6,7,8,9]:
    wkmap = pd.Series(train[train['Week_num'] == j].set_index('ID')['adjusted_demand'], index=train['ID'].unique()).to_dict()
    trainIDdf[f'Wk_{j}_dem'] = trainIDdf['ID'].map(wkmap)

trainIDdf['ID'] = trainIDdf['ID'].astype('category')
trainIDdf['Client_ID'] = trainIDdf['Client_ID'].astype('category')
trainIDdf['Product_ID'] = trainIDdf['Product_ID'].astype('category')


trainIDdf.head()

,ID,Client_ID,Product_ID,ccid_mean,ccid_median,cpid_mean,cpid_median,Wk_6_dem,Wk_7_dem,Wk_8_dem,Wk_9_dem
0,0,15766,325,5.51,4.0,1.83,1.0,NaN,NaN,NaN,NaN
1,1,15766,328,5.51,4.0,1.88,2.0,NaN,NaN,NaN,NaN
2,2,15766,1212,5.51,4.0,2.51,2.0,NaN,NaN,4.0,1.0
3,3,15766,1216,5.51,4.0,2.16,2.0,1.0,2.0,5.0,NaN
4,4,15766,1220,5.51,4.0,3.10,3.0,NaN,NaN,1.0,NaN


In [ ]:
# X_train.drop(['Wk_3_dem', 'Wk_4_dem','Wk_5_dem'], axis=1, inplace=True)
X_train.rename(columns={'Wk_6_dem': 'lag_3', 'Wk_7_dem': 'lag_2', 'Wk_8_dem': 'lag_1'}, inplace=True)

features = ['Client_ID', 'Product_ID', 
            'ccid_mean', 'ccid_median', 
            'cpid_mean', 'cpid_median',
            'lag_3', 'lag_2', 'lag_1']
target = ['Wk_9_dem']

X_train = trainIDdf.dropna(subset=['Wk_9_dem']).loc[features]

# trainIDdf['Wk_9_dem'].fillna(trainIDdf['ccid_mean'], inplace=True)
y_train = trainIDdf.dropna(subset=['Wk_9_dem']).loc[target]

X_train.head()

,Client_ID,Product_ID,ccid_mean,ccid_median,cpid_mean,cpid_median,lag_3,lag_2,lag_1
2,15766,1212,5.51,4.0,2.51,2.0,NaN,NaN,4.0
5,15766,1238,5.51,4.0,3.38,3.0,2.0,2.0,3.0
6,15766,1240,5.51,4.0,4.55,4.0,NaN,8.0,2.0
7,15766,1242,5.51,4.0,2.86,3.0,3.0,2.0,1.0
8,15766,1250,5.51,4.0,6.79,6.0,1.0,14.0,8.0


In [7]:
del train

## XGBoost Estimation

The test data contains observains for week 10 and 11. We exploit a two-step demand prediction model to address this format. First, we estimate the demand in week 10 using demand in week 9 as our target variables. Next, we estimate the demand in week 11 using the predicted demand in week 10 as our target variable.

### Grid Search with Cross Validation
To find the best hyperparameter for the XGBoost model, we use a grid search algorthim. To avoid overfitting the model to our data, we use a 5 fold cross-validation. Note that since we're using the adjusted demand in week 9 as our target variable, we're abstracting from the time dimension. Therefore, our cross-validation does not lead to data leakage.

In [ ]:
# ** Grid Search Code Block**

### Demand Prediction using XGBoost
Using the hyperparameters found in the previous step, we fit our modified training data using an XGBoost ensemble. 

In [8]:
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)

model = xgb.train({'max_depth': 8, 'eta': 0.1, 'objective': 'reg:squaredlogerror'}, dtrain, num_boost_round=250)

test_pred = model.predict(xgb.DMatrix(X_train, enable_categorical=True))
test_pred[test_pred < 0] = 0

print(rmsle(test_pred, y_train))

0.4553438057287803



We merge the predictions for week 10 with the test data set.

In [9]:
# get ccid, cpid means and medians into test data
cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(test['ccid'].unique().tolist())].set_index('ccid')['adj_dem_mean'], index=client_stats[client_stats['ccid'].isin(test['ccid'].unique().tolist())]['ccid']).to_dict()
test['ccid_mean'] = test['ccid'].map(cidmapping)

cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(test['ccid'].unique().tolist())].set_index('ccid')['adj_dem_median'], index=client_stats[client_stats['ccid'].isin(test['ccid'].unique().tolist())]['ccid']).to_dict()
test['ccid_median'] = test['ccid'].map(cidmapping)

# cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(test['ccid'].unique().tolist())].set_index('ccid')['mean_pct'], index=client_stats[client_stats['ccid'].isin(test['ccid'].unique().tolist())]['ccid']).to_dict()
# test['ccid_mean_pct'] = test['ccid'].map(cidmapping).round(3)

pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(test['cpid'].unique().tolist())].set_index('cpid')['adj_dem_mean'], index=product_stats[product_stats['cpid'].isin(test['cpid'].unique().tolist())]['cpid']).to_dict()
test['cpid_mean'] = test['cpid'].map(pidmapping)

pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(test['cpid'].unique().tolist())].set_index('cpid')['adj_dem_median'], index=product_stats[product_stats['cpid'].isin(test['cpid'].unique().tolist())]['cpid']).to_dict()
test['cpid_median'] = test['cpid'].map(pidmapping)

del cidmapping, pidmapping

test = test[['id', 'ID', 'Week_num', 'Client_ID', 'Product_ID', 'ccid_mean', 'ccid_median', 'cpid_mean', 'cpid_median']].sort_values(by='id')
test.reset_index(drop=True, inplace=True)


# get adjusted demand from previous weeks
lagmap = pd.Series(trainIDdf[['ID', 'Wk_7_dem']].set_index('ID')['Wk_7_dem'], index=trainIDdf['ID'].tolist()).to_dict()
test['lag_3'] = test['ID'].map(lagmap)

lagmap = pd.Series(trainIDdf[['ID', 'Wk_8_dem']].set_index('ID')['Wk_8_dem'], index=trainIDdf['ID'].tolist()).to_dict()
test['lag_2'] = test['ID'].map(lagmap)

lagmap = pd.Series(trainIDdf[['ID', 'Wk_9_dem']].set_index('ID')['Wk_9_dem'], index=trainIDdf['ID'].tolist()).to_dict()
test['lag_1'] = test['ID'].map(lagmap)

del lagmap


test['Client_ID'] = test['Client_ID'].astype('category')
test['Product_ID'] = test['Product_ID'].astype('category')

test.head()

,id,ID,Week_num,Client_ID,Product_ID,ccid_mean,ccid_median,cpid_mean,cpid_median,lag_3,lag_2,lag_1
0,0,25973294,11,4639078,35305,2.73,2.0,4.79,4.0,NaN,NaN,4.0
1,1,23662849,11,4705135,1238,3.47,2.0,2.07,2.0,NaN,NaN,NaN
2,2,21257171,10,4549769,32940,6.78,4.0,2.45,2.0,NaN,2.0,2.0
3,3,5334985,11,4717855,43066,2.12,1.0,1.33,1.0,NaN,NaN,NaN
4,4,4150753,11,966351,1277,5.24,5.0,NaN,NaN,NaN,NaN,NaN


First, we estimate the demand in week 10 using demand in week 9 as our target variables. Next, we estimate the demand in week 11 using the predicted demand in week 10 as our target variable.

In [ ]:
# week 10 test data then make the prediction
X_test_wk10 = xgb.DMatrix(test[test['Week_num'] == 10].loc[features], enable_categorical=True)
predictions_10 = model.predict(X_test_wk10)


# add week 10 prediction to test dataframe
test['wk_10_pred_dem'] = np.nan
test.loc[test['Week_num'] == 10, 'wk_10_pred_dem'] = predictions_10


# week 11 test data then make the prediction
test_wk11 = test[test['Week_num'] == 11].copy(deep=True)
test_wk11.drop(['lag_3'], axis=1, inplace=True)
test_wk11.rename(columns={'lag_2': 'lag_3', 'lag_1': 'lag_2', 'wk_10_pred_dem': 'lag_1'}, inplace=True)
X_test_wk11 = xgb.DMatrix(test_wk11.loc[features], enable_categorical=True)

predictions_11 = model.predict(X_test_wk11)

# add week 11 prediction to test dataframe
test['wk_11_pred_dem'] = np.nan
test.loc[test['Week_num'] == 11, 'wk_11_pred_dem'] = predictions_11

test.head()

,id,ID,Week_num,Client_ID,Product_ID,ccid_mean,ccid_median,cpid_mean,cpid_median,lag_3,lag_2,lag_1,wk_10_pred_dem,wk_11_pred_dem
0,0,25973294,11,4639078,35305,2.73,2.0,4.79,4.0,NaN,NaN,4.0,NaN,2.798304
1,1,23662849,11,4705135,1238,3.47,2.0,2.07,2.0,NaN,NaN,NaN,NaN,1.405669
2,2,21257171,10,4549769,32940,6.78,4.0,2.45,2.0,NaN,2.0,2.0,1.987087,NaN
3,3,5334985,11,4717855,43066,2.12,1.0,1.33,1.0,NaN,NaN,NaN,NaN,1.022621
4,4,4150753,11,966351,1277,5.24,5.0,NaN,NaN,NaN,NaN,NaN,NaN,4.480295


In [ ]:
# make the submission file!
# Kaggle score is 0.49323

submission = pd.DataFrame()

submission['id'] = np.arange(len(test))
submission['Demanda_uni_equil'] = test['wk_10_pred_dem'].combine_first(test['wk_11_pred_dem'])
submission.loc[submission['Demanda_uni_equil'] < 0, 'Demanda_uni_equil'] = 0

submission.to_csv("xgb_prediction_final_1.csv", index=False)

submission.head()

,id,Demanda_uni_equil
0,0,2.798304
1,1,1.405669
2,2,1.987087
3,3,1.022621
4,4,4.480295


!!! Questions and Notes

What's the share of missing observations in the training data i.e. what share of observations do not have week 9?
What's the share of missing observations in the test data? 
Note that we do not impute the demand for missing clients or products in our training data. Moreover, we focus our prediction using adjusted demand in week 9 as our target variable. These two assumption may lead to a higher/lower score.